In [44]:
import pandas as pd
import numpy as np

In [45]:
classes: set[str] = {"setosa", "versicolor", "virginica"}
measurements_columns_names: list[str] = ["sepallength", "sepalwidth", "petallength", "petalwidth"]
iris_data: pd.DataFrame = pd.read_csv(
	"iris.csv",
	na_values={"NA", "-", "n/a", "na"},
	dtype={
		"sepallength": np.float64,
		"sepalwidth": np.float64,
		"petallength": np.float64,
		"petalwidth": np.float64,
		"class": pd.CategoricalDtype(categories=classes),
	}
)
iris_data

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [46]:
from sklearn.preprocessing import StandardScaler
std_iris_data: pd.DataFrame = iris_data.copy()
scaler: StandardScaler = StandardScaler()
for columnName in measurements_columns_names:
	std_iris_data[columnName] = scaler.fit_transform(iris_data[columnName].values.reshape(-1, 1))

std_iris_data


,sepallength,sepalwidth,petallength,petalwidth,class
0,-0.900681,1.032057,-1.341272,-1.312977,setosa
1,-1.143017,-0.124958,-1.341272,-1.312977,setosa
2,-1.385353,0.337848,-1.398138,-1.312977,setosa
3,-1.506521,0.106445,-1.284407,-1.312977,setosa
4,-1.021849,1.263460,-1.341272,-1.312977,setosa
...,...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956,virginica
146,0.553333,-1.281972,0.705893,0.922064,virginica
147,0.795669,-0.124958,0.819624,1.053537,virginica
148,0.432165,0.800654,0.933356,1.447956,virginica


In [47]:
std_iris_data.corr()

,sepallength,sepalwidth,petallength,petalwidth
sepallength,1.000000,-0.109369,0.871754,0.817954
sepalwidth,-0.109369,1.000000,-0.420516,-0.356544
petallength,0.871754,-0.420516,1.000000,0.962757
petalwidth,0.817954,-0.356544,0.962757,1.000000


In [48]:
std_iris_data_corr_eigenvalues, _ = np.linalg.eig(std_iris_data.corr().values)
std_iris_data_corr_eigenvalues.sort()
std_iris_data_corr_eigenvalues = std_iris_data_corr_eigenvalues[::-1]
std_iris_data_corr_eigenvalues

# resources:
# https://towardsdatascience.com/principal-component-analysis-ac90b73f68f5
# https://towardsdatascience.com/feature-extraction-using-principal-component-analysis-a-simplified-visual-demo-e5592ced100a

print("explained variance in relation to the number of features:")
for i in range(len(std_iris_data_corr_eigenvalues)):
	explained_variance: float = sum(std_iris_data_corr_eigenvalues[:i+1]) \
		/ sum(std_iris_data_corr_eigenvalues)
	print(f"{i+1} ftr: {100 * explained_variance:.2f}%")
# it is possible to remove 2 columns without huge information loss

explained variance in relation to the number of features:
1 ftr: 72.77%
2 ftr: 95.80%
3 ftr: 99.48%
4 ftr: 100.00%


In [49]:
from sklearn.decomposition import PCA
pca: PCA = PCA(n_components=2)
principal_components = pca.fit_transform(
	std_iris_data[measurements_columns_names].values
)
principal_df: pd.DataFrame = pd.DataFrame(
	data=principal_components,
	columns=["pc 1", "pc 2"],
)
principal_df = pd.concat([principal_df, std_iris_data["class"]], axis=1)
principal_df

,pc 1,pc 2,class
0,-2.264542,0.505704,setosa
1,-2.086426,-0.655405,setosa
2,-2.367950,-0.318477,setosa
3,-2.304197,-0.575368,setosa
4,-2.388777,0.674767,setosa
...,...,...,...
145,1.870522,0.382822,virginica
146,1.558492,-0.905314,virginica
147,1.520845,0.266795,virginica
148,1.376391,1.016362,virginica
